In [ ]:
'''
*********************************
***  Name: Truong Nhat Hoang  ***
***  Student ID: 51703092     ***
***  HMM POS Tagger           ***
***  NLP - 504045             ***
*********************************
'''

In [31]:
from utils import hmm_model
from utils.data_loader import DataLoader
from utils.hmm_model import HMMBasic

In [53]:
dloader = DataLoader('./data/')
dloader.load()
train_word = [j for i in dloader.data_train['label'] for j in i].__len__()
test_word = [j for i in dloader.data_test['label'] for j in i].__len__()

 ./data/CR/CR08.POS
All data is loaded!!


In [54]:
hmm_basic = HMMBasic()
hmm_basic.train(dloader.data_train)
score = hmm_basic.eval(dloader.data_test)

In [94]:
emiss_prob = {
    key: {
        sub_key_val[0]: sub_key_val[1]
        for sub_key_val in val
    } 
    for key, val in hmm_basic.dict2_word_tag.items()
}

trans_prob = {
    key: {
        sub_key_val[0]: sub_key_val[1]
        for sub_key_val in val
    } 
    for key, val in hmm_basic.dict2_tag_follow_tag_.items()
}

In [101]:
set_type = set()
for key in emiss_prob:
    set_type.add(type(emiss_prob[key]))

In [109]:
emiss_prob.get('IN', 0)

0

In [48]:
start_tag = dict()
sum_all = 0
for sen in dloader.processed_data:
    sum_all += 1
    start_tag[sen[0]['tag']] = start_tag.get(sen[0]['tag'], 0) + 1
    
start_tag = {key: val/sum_all for key, val in start_tag.items()}
# start_tag = {key: start_tag[key] if key in start_tag else 0 for key in hmm_basic.train_li_tags}

In [171]:
def viterbi(_obs, _states, _s_pro, _t_pro, _e_pro):
    # init
    path = {
        s: list()
        for s in _states
    }
    curr_pro = dict()
    for s in _states:
        curr_pro[s] = _s_pro.get(s, 1) * _e_pro.get(s, dict()).get(obs[0], 1)
    for i in range(1, len(obs)):
        last_pro = curr_pro
        curr_pro = {}
        for curr_state in _states:
            max_pro, last_sta = max((((
                        last_pro[last_state] 
                         * _t_pro.get(last_state, dict()).get(curr_state, 1) 
                         * _e_pro.get(curr_state, dict()).get(obs[i], 1)
                    ),
                    last_state
                )
                for last_state in _states
            ), key=lambda x: x[0])
            

            curr_pro[curr_state] = max_pro
            path[curr_state].append(last_sta)
        print(curr_pro)

    max_pro = -1
    max_path = None
    for s in _states:
        path[s].append(s)
        if curr_pro[s] > max_pro:
            max_path = path[s]
            max_pro = curr_pro[s]
    return max_path

In [172]:
hmm_basic.train_li_tags[:5]

['IN', 'VBD', 'VBD', 'IN', 'NNP']

In [173]:
resut_vtb = viterbi(
    ['My', 'name', 'is', 'Hoang'],
    list(set(hmm_basic.train_li_tags)),
    start_tag,
    trans_prob,
    emiss_prob
)

{'': 0.19361710294004758, 'EX': 0.018404083825900055, 'SYM': 0.1729484915943809, 'NNP|NPS': 0.19361710294004758, 'NNP': 0.025553201313836024, '.': 0.014738619789667614, 'PRP': 0.014122679292327711, 'VBG': 0.0035458811363409414, 'CD|NN|NP': 0.19361710294004758, 'JJ|RB': 0.19361710294004758, 'NN|VBG': 0.08591770937284102, 'NNP|NN': 0.19361710294004758, 'JJ': 0.04450031214132649, 'IN|RP': 0.19361710294004758, 'MD|VB': 0.19361710294004758, 'NN|SYM': 0.19361710294004758, 'TO': 0.006831964381668842, 'NNPS': 0.014738619789667614, 'PDT': 0.19361710294004758, 'NN|JJ': 0.1729484915943809, 'VBZ': 0.016453428682088378, 'WP|MD|NP': 0.19361710294004758, 'UH': 0.018404083825900055, 'DT|RB': 0.19361710294004758, 'VBG|NN': 0.1729484915943809, 'VBD': 0.05755498982567102, ',': 0.07522837184309511, 'RP|IN': 0.19361710294004758, 'CD|NNS': 0.19361710294004758, 'JJ|VBN': 0.19361710294004758, 'VB': 0.007091435733834779, 'CD': 0.014738619789667614, 'RP': 0.07522837184309511, 'CD|NN': 0.1729484915943809, 'RBR|N

In [174]:
resut_vtb

['DT', '', '']

In [ ]:
.        self.train_li_words = list()
        self.train_li_tags = list()
        self.dict2_tag_follow_tag_ = dict()
        self.dict2_word_tag = dict()
        self.dict_word_tag_baseline = dict()

In [ ]:
resut_vtb = viterbi(
    hmm_basic.train_li_words,
    hmm_basic.train_li_tags,
    start_probability,
    transition_probability,
    emission_probability
)

In [114]:
states = (
    'Healthy',
    'Fever'
)
observations = (
    'normal',
    'cold',
    'dizzy'
)

start_probability = {
    'Healthy': 0.6,
    'Fever': 0.4
}

transition_probability = {
    'Healthy': {'Healthy': 0.7, 'Fever': 0.3},
    'Fever': {'Healthy': 0.4, 'Fever': 0.6},
}

emission_probability = {
    'Healthy': {
        'normal': 0.5,
        'cold': 0.4,
        'dizzy': 0.1
    },
    'Fever': {
        'normal': 0.1,
        'cold': 0.3,
        'dizzy': 0.6
    },
}

obs = [
    'normal',
    'cold',
    'dizzy'
]

resut_vtb = viterbi(
    obs,
    states,
    start_probability,
    transition_probability,
    emission_probability
)

print(resut_vtb)

['Healthy', 'Healthy', 'Fever']
